In [1]:
import json
import numpy
from collections import defaultdict
from PIL import Image
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import time

In [2]:
import os
import baseline
import pandas as pd
import numpy as np
#TEMPORARILY SUPPRESS WARNINGS
import warnings

warnings.filterwarnings("ignore")

# get the actual labels in the validation set

In [3]:
captions_train_path = "/mnt/muthderd/MIDS/W210/data/annotations_trainval2017/annotations/captions_train2017.json"
captions_val_path = "/mnt/muthderd/MIDS/W210/data/annotations_trainval2017/annotations/captions_val2017.json"
#captions_train  = json.loads(open(captions_train_path).read())

In [4]:
inst_train_path = "/mnt/muthderd/MIDS/W210/data/annotations_trainval2017/annotations/instances_train2017.json"
inst_val_path = "/mnt/muthderd/MIDS/W210/data/annotations_trainval2017/annotations/instances_val2017.json"
#inst_train  = json.loads(open(inst_train_path).read())

In [5]:
def get_electronics_info(captions_path, inst_path):
    captions  = json.loads(open(captions_path).read())
    inst  = json.loads(open(inst_path).read())
    image_df = pd.DataFrame(inst['images'])
    annotation_df = pd.DataFrame(inst['annotations'])
    #the 'id' in image_df needs to be changed to 'image_id' in order to join with annotations_df
    renamed_image_df = image_df.copy(deep=True)
    renamed_image_df.rename(columns={'id':'image_id'}, inplace=True)
    images_and_annotations_df = annotation_df.merge(renamed_image_df,on='image_id', how='left')
    electronics_only_merged_df = images_and_annotations_df.loc[(images_and_annotations_df['category_id'] >= 72) & (images_and_annotations_df['category_id'] <= 77)]
    
    #select all images that contain electronics
    #all_images_with_electronics = list(pd.Series(electronics_only_merged_df['image_id']).unique())
    
    return(electronics_only_merged_df)

In [6]:
train_electronics = get_electronics_info(captions_train_path, inst_train_path)
val_electronics = get_electronics_info(captions_val_path, inst_val_path)

In [7]:
#train_electronics.head()

In [8]:
#val_electronics.head()

In [55]:
val_electronics[val_electronics['image_id']==139]

,area,bbox,category_id,id,image_id,iscrowd,segmentation,coco_url,date_captured,file_name,flickr_url,height,license,width
24452,13244.65770,"[7.03, 167.76, 149.32, 94.87]",72,34646,139,0,"[[9.66, 167.76, 156.35, 173.04, 153.71, 256.48...",http://images.cocodataset.org/val2017/00000000...,2013-11-21 01:34:01,000000000139.jpg,http://farm9.staticflickr.com/8035/8024364858_...,426,2,640
24458,5833.11795,"[557.21, 209.19, 81.35, 78.73]",72,35802,139,0,"[[563.33, 209.19, 637.69, 209.19, 638.56, 287....",http://images.cocodataset.org/val2017/00000000...,2013-11-21 01:34:01,000000000139.jpg,http://farm9.staticflickr.com/8035/8024364858_...,426,2,640


In [10]:
len(val_electronics[val_electronics['image_id']==464476]['bbox'])

2

In [11]:
np.array([[coord for coord in entry] for entry in val_electronics[val_electronics['image_id']==464476]['bbox']]).reshape(1,2,4)

array([[[279.11, 209.81, 206.02, 160.44],
        [281.13,  40.81, 165.48, 166.35]]])

In [12]:
np.array(val_electronics[val_electronics['image_id']==464476]['bbox'].apply(np.array)).reshape(1,2,4)

ValueError: cannot reshape array of size 2 into shape (1,2,4)

In [131]:
val_electronics[val_electronics['image_id']==464476]['category_id']
np.array([entry for entry in val_electronics[val_electronics['image_id']==464476]['category_id']]).reshape(1,2)

array([[72, 72]])

In [13]:
#extract electronics IDs in each image
eIDs_in_images = val_electronics.groupby('image_id')['category_id'].apply(list).to_dict()
#select all images that contain electronics
train_images_with_electronics = list(pd.Series(train_electronics['image_id']).unique())
val_images_with_electronics = list(pd.Series(val_electronics['image_id']).unique())

In [14]:
#val_electronics

In [15]:
#val_images_with_electronics[:10]

In [16]:
len(val_images_with_electronics)

597

# get the validation images with electronics

In [17]:
val_folder_name = '/mnt/muthderd/MIDS/W210/data/val2017/'
all_val_filenames = os.listdir(val_folder_name)

In [18]:
max_file_id_length = len(all_val_filenames[0]) - 4#subtract 4 for the '.jpg' suffix

In [19]:
files_to_open = []
for image_id in val_images_with_electronics:
    file_id = str(image_id)
    zeros_to_add = max_file_id_length-len(file_id)
    filename = ('0'*zeros_to_add) + file_id + '.jpg'
    files_to_open.append(filename)

In [20]:
#files_to_open[:10]

In [23]:
from gluoncv import data as gdata
from gluoncv import utils as gutils

In [24]:
map_data = gdata.COCODetection(root='/mnt/muthderd/MIDS/W210/data/', splits='/mnt/muthderd/MIDS/W210/data/annotations_trainval2017/annotations/instances_val2017')

loading annotations into memory...
Done (t=0.22s)
creating index...
index created!


In [25]:
dir(map_data)

['CLASSES',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_check_load_bbox',
 '_coco',
 '_items',
 '_labels',
 '_load_jsons',
 '_min_object_area',
 '_root',
 '_skip_empty',
 '_splits',
 '_transform',
 '_use_crowd',
 'classes',
 'coco',
 'contiguous_id_to_json',
 'index_map',
 'json_id_to_contiguous',
 'num_class',
 'transform',
 'transform_first']

In [26]:
type(map_data)

gluoncv.data.mscoco.detection.COCODetection

In [27]:
coco_metric = gutils.metrics.COCODetectionMetric(map_data, save_prefix='/mnt/muthderd/MIDS/W210/data/annotations_trainval2017/annotations/instances_val2017')

In [28]:
#coco_metric.get()

In [29]:
map_object = gutils.metrics.VOCMApMetric()

In [52]:
gt_bboxes#.reshape(2,4)

array([[[279.11, 209.81, 206.02, 160.44],
        [281.13,  40.81, 165.48, 166.35]]])

In [48]:
bounding_boxes.reshape(100,4)


[[ 70.859024  81.128845 162.25264  329.08127 ]
 [376.34207  282.3346   660.26355  518.43146 ]
 [376.49896  289.02313  657.53925  509.74353 ]
 [194.30513   31.84256  372.9276   205.11937 ]
 [194.30513   31.84256  372.9276   205.11937 ]
 [206.42798   34.58731  375.59808  202.87526 ]
 [ 27.974426  10.990707 389.3849   263.2132  ]
 [374.799     49.53401  609.736    288.7542  ]
 [ 72.88392  173.17085   97.29386  224.00603 ]
 [376.34207  282.3346   660.26355  518.43146 ]
 [ 78.94175  195.10974   91.47288  225.88666 ]
 [289.45917   90.62057  296.83038  109.50909 ]
 [376.34207  282.3346   660.26355  518.43146 ]
 [ -1.        -1.        -1.        -1.      ]
 [ -1.        -1.        -1.        -1.      ]
 [ -1.        -1.        -1.        -1.      ]
 [ -1.        -1.        -1.        -1.      ]
 [ -1.        -1.        -1.        -1.      ]
 [ -1.        -1.        -1.        -1.      ]
 [ -1.        -1.        -1.        -1.      ]
 [ -1.        -1.        -1.        -1.      ]
 [ -1.      

In [45]:
#[minx, miny, maxx, maxy]

{'keys': [1, 0, 2, 3]}

In [ ]:
def get_iou(a, b, epsilon=1e-5):
    """ Given two boxes `a` and `b` defined as a list of four numbers:
            [x1,y1,x2,y2]
        where:
            x1,y1 represent the upper left corner
            x2,y2 represent the lower right corner
        It returns the Intersect of Union score for these two boxes.

    Args:
        a:          (list of 4 numbers) [x1,y1,x2,y2]
        b:          (list of 4 numbers) [x1,y1,x2,y2]
        epsilon:    (float) Small value to prevent division by zero

    Returns:
        (float) The Intersect of Union score.
    """
    # COORDINATES OF THE INTERSECTION BOX
    x1 = max(a[0], b[0])
    y1 = max(a[1], b[1])
    x2 = min(a[2], b[2])
    y2 = min(a[3], b[3])

    # AREA OF OVERLAP - Area where the boxes intersect
    width = (x2 - x1)
    height = (y2 - y1)
    # handle case where there is NO overlap
    if (width<0) or (height <0):
        return 0.0
    area_overlap = width * height

    # COMBINED AREA
    area_a = (a[2] - a[0]) * (a[3] - a[1])
    area_b = (b[2] - b[0]) * (b[3] - b[1])
    area_combined = area_a + area_b - area_overlap

    # RATIO OF AREA OF OVERLAP OVER COMBINED AREA
    iou = area_overlap / (area_combined+epsilon)
    return iou

In [53]:
labels

,area,bbox,category_id,id,image_id,iscrowd,segmentation,coco_url,date_captured,file_name,flickr_url,height,license,width
48,31935.48505,"[279.11, 209.81, 206.02, 160.44]",72,29131,464476,0,"[[279.11, 370.25, 281.96, 214.56, 283.86, 209....",http://images.cocodataset.org/val2017/00000046...,2013-11-14 18:23:40,000000464476.jpg,http://farm1.staticflickr.com/24/35275654_7d93...,375,3,500
2863,22237.67500,"[281.13, 40.81, 165.48, 166.35]",72,1624484,464476,0,"[[283.29, 44.28, 288.06, 40.81, 441.84, 41.68,...",http://images.cocodataset.org/val2017/00000046...,2013-11-14 18:23:40,000000464476.jpg,http://farm1.staticflickr.com/24/35275654_7d93...,375,3,500


# test models

In [42]:
%%capture

coco_metric.reset()

#results_folder = '/mnt/muthderd/MIDS/W210/data/val_results_2/'
results_folder = '/mnt/muthderd/MIDS/W210/data/val_results/'
data_folder = '/mnt/muthderd/MIDS/W210/data/val2017/'

#test_filenames = os.listdir(data_folder)
test_filenames = files_to_open[:25]

all_test_annotations = pd.DataFrame(columns=['image_id', 'class_id', 'scores', 1, 2, 3, 4])

#baseline_model = baseline.BaselineClassifier(model='faster_rcnn_resnet101_v1d_coco')
baseline_model = baseline.BaselineClassifier(model='yolo3_darknet53_coco')

pred_start = time.time()

for image in test_filenames:
    class_ids, scores, bounding_boxes = baseline_model.classify_objects(image, data_folder, results_folder)
    
    class_id_series = pd.Series(class_ids[0].asnumpy().reshape(len(class_ids[0]),))
    score_series = pd.Series(scores[0].asnumpy().reshape(len(scores[0]),))
    b_box_DF = pd.DataFrame(bounding_boxes[0].asnumpy().reshape(len(bounding_boxes[0]),4), columns=[1,2,3,4])
    
    class_id_series[class_id_series>-1]
    score_series[class_id_series>-1]
    b_box_DF[class_id_series>-1]
    
    test_annotations = b_box_DF[class_id_series>-1]
    test_annotations['class_id'] = class_id_series[class_id_series>-1]
    test_annotations['scores'] = score_series[class_id_series>-1]
    
    test_annotations['image_id'] = [image[:-4]]*len(test_annotations)
    test_annotations = test_annotations[['image_id', 'class_id', 'scores', 1, 2, 3, 4]]
    
    all_test_annotations = pd.concat([all_test_annotations, test_annotations], ignore_index=True)
    
    #image_id = int(image[:-4])
    #labels = val_electronics[val_electronics['image_id']==image_id]
    #gt_bboxes = np.array([[coord for coord in entry] for entry in labels['bbox']]).reshape(1,len(labels),4)
    ##gt_bboxes = np.array(labels['bbox'].apply(np.array))
    #gt_labels = np.array([entry for entry in labels['category_id']]).reshape(1,len(labels))
    ##gt_labels = np.array(labels['category_id'].apply(np.array))
    
    #map_object.update(bounding_boxes, class_ids.reshape(1,100), scores.reshape(1,100), gt_bboxes, gt_labels)
    coco_metric.update(bounding_boxes, class_ids, scores)
    
print("Total Annotations: ", str(len(all_test_annotations)))

pred_end = time.time()
pred_duration = pred_end - pred_start

ValueError: zero-dimensional arrays cannot be concatenated

In [153]:
#map_object.get()
coco_metric.get()

Loading and preparing results...
DONE (t=0.00s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=4.52s).
Accumulating evaluation results...
DONE (t=0.92s).


(['~~~~ Summary metrics ~~~~\n',
  'person',
  'bicycle',
  'car',
  'motorcycle',
  'airplane',
  'bus',
  'train',
  'truck',
  'boat',
  'traffic light',
  'fire hydrant',
  'stop sign',
  'parking meter',
  'bench',
  'bird',
  'cat',
  'dog',
  'horse',
  'sheep',
  'cow',
  'elephant',
  'bear',
  'zebra',
  'giraffe',
  'backpack',
  'umbrella',
  'handbag',
  'tie',
  'suitcase',
  'frisbee',
  'skis',
  'snowboard',
  'sports ball',
  'kite',
  'baseball bat',
  'baseball glove',
  'skateboard',
  'surfboard',
  'tennis racket',
  'bottle',
  'wine glass',
  'cup',
  'fork',
  'knife',
  'spoon',
  'bowl',
  'banana',
  'apple',
  'sandwich',
  'orange',
  'broccoli',
  'carrot',
  'hot dog',
  'pizza',
  'donut',
  'cake',
  'chair',
  'couch',
  'potted plant',
  'bed',
  'dining table',
  'toilet',
  'tv',
  'laptop',
  'mouse',
  'remote',
  'keyboard',
  'cell phone',
  'microwave',
  'oven',
  'toaster',
  'sink',
  'refrigerator',
  'book',
  'clock',
  'vase',
  'sciss

In [138]:
pred_duration

884.2778913974762

In [100]:
all_test_annotations.head()

,image_id,class_id,scores,1,2,3,4
0,000000464476,0.0,0.927936,70.859024,81.128845,162.252640,329.081268
1,000000464476,63.0,0.692063,376.342072,282.334595,660.263550,518.431458
2,000000464476,62.0,0.356848,376.498962,289.023132,657.539246,509.743530
3,000000464476,7.0,0.326432,194.305130,31.842560,372.927612,205.119370
4,000000464476,6.0,0.145672,194.305130,31.842560,372.927612,205.119370


In [139]:
#all_test_annotations.to_csv('val_result_annotations_2.csv')
all_test_annotations.to_csv('val_result_annotations.csv')

In [140]:
pred_prob_threshold = 0.5
result_annotations = all_test_annotations[all_test_annotations['scores']>pred_prob_threshold]
eIDs_in_results = result_annotations.groupby('image_id')['class_id'].apply(list).to_dict()
#eIDs_in_results

In [141]:
#quick hack to deal with the fact that the class IDs predicted by the baseline model are 10 lower than those in the annotation file
#and to throw out non electronics identified
predicted_eIDs = {}
for image_id in eIDs_in_results.keys():
    corrected_IDs = np.add(eIDs_in_results[image_id],10).astype(int)
    llimit = corrected_IDs >= 72
    ulimit = corrected_IDs <=77
    #print(llimit & ulimit)
    corrected_IDs = corrected_IDs[llimit & ulimit]
    #eIDs_in_results[image_id] = list(corrected_IDs)
    predicted_eIDs[int(image_id)] = list(corrected_IDs)
#predicted_eIDs

In [142]:
total_pred_counts = []#[0, 0, 0, 0, 0, 0]#can use a smarter structure here
total_label_counts = []#[0, 0, 0, 0, 0, 0]
#detection_accuracies = []#[0, 0, 0, 0, 0, 0]
#detection_accuracies_by_class = []
num_labels_detected = []
num_labels = []
unique_labels_by_image = []
unique_preds_by_image = []
#current counting method will not take into account false positives
for image_id in predicted_eIDs.keys():
    current_unique_preds = [0, 0, 0, 0, 0, 0]
    current_unique_labels = [0, 0, 0, 0, 0, 0]
    pred_counts = [0, 0, 0, 0, 0, 0]#can use a smarter structure here
    label_counts = [0, 0, 0, 0, 0, 0]
    
    labels = eIDs_in_images[image_id]
    for label in labels:
        label_counts[label-72] += 1
    preds = predicted_eIDs[image_id]
    for pred in preds:
        pred_counts[pred-72] += 1
    
    unique_labels = list(set(labels))
    unique_preds = list(set(preds))
    unique_label_count = len(unique_labels)
    unique_labels_detected = 0
    for label in unique_labels:
        #label_counts[label-72] += 1
        current_unique_labels[label-72] += 1
        if label in unique_preds:
            unique_labels_detected += 1
    for pred in unique_preds:
        #pred_counts[pred-72] += 1
        current_unique_preds[pred-72] += 1
    
    #new method: count instances of each id to take FP into account
    
    #new method: tally accuracy for each image separately
    
    #store counts of each label and pred for each image
    total_pred_counts.append(pred_counts)
    total_label_counts.append(label_counts)
    
    #store unique labels and preds
    unique_labels_by_image.append(current_unique_labels)
    unique_preds_by_image.append(current_unique_preds)
    
    #counts of unique labels and preds
    num_labels.append(unique_label_count)
    num_labels_detected.append(unique_labels_detected)
    
    #detection_accuracies.append(num_labels_detected/num_labels)
    #detection_accuracies_by_class.append(list(np.divide(current_pred_counts, current_label_counts)))

In [143]:
pred_counts = pd.DataFrame(total_pred_counts)
label_counts = pd.DataFrame(total_label_counts)
pred_counts[6] = pred_counts.sum(axis=1)
label_counts[6] = label_counts.sum(axis=1)

In [42]:
label_counts.head()

,0,1,2,3,4,5,6
0,1,0,0,0,0,0,1
1,1,0,0,0,0,0,1
2,1,0,0,0,0,0,1
3,1,0,0,0,0,0,1
4,2,0,0,0,1,0,3


In [144]:
errors = label_counts.subtract(pred_counts).abs()
error_rate_by_class = errors.sum(axis=0).divide(label_counts.sum(axis=0))
error_rate_by_class

0    0.431579
1    0.366071
2    0.314286
3    0.709091
4    0.543624
5    0.630522
6    0.476301
dtype: float64

0    0.52
1    0.50
2    0.00
3    0.00
4    0.25
5    0.50
6    0.40
dtype: float64

In [47]:
#np.divide(num_labels_detected, num_labels)

array([1. , 1. , 0. , 1. , 0.5, 1. , 1. , 1. , 1. , 1. , 1. , 0. , 1. ,
       1. , 0. , 0. , 0. , 0. , 1. , 1. , 1. , 0. , 1. , 1. ])

In [145]:
#average percentage of unique electronics labels identified in a picture
np.mean(np.divide(num_labels_detected, num_labels))

0.5712962962962963

In [146]:
num_images = len(num_labels)
num_images

576

In [147]:
#which of the unique labels were predicted?
unique_labels_detected = (pd.DataFrame(unique_labels_by_image).astype(bool) & pd.DataFrame(unique_preds_by_image).astype(bool)).multiply(1).astype(int)
unique_labels_detected.sum(axis=0)

0    141
1    124
2     65
3     68
4     66
5     88
dtype: int64

In [148]:
num_images_labeled = pd.DataFrame(unique_labels_by_image).sum(axis=0)
num_images_labeled

0    204
1    176
2     87
3    138
4    103
5    205
dtype: int64

In [149]:
#overall rate of detecting that a label is present
unique_labels_detected.sum(axis=0).sum()/num_images_labeled.sum()

0.6046002190580504

In [150]:
#for each class, what fraction of images containing that class are predicted to contain that class?
unique_labels_detected.sum(axis=0).divide(num_images_labeled)

0    0.691176
1    0.704545
2    0.747126
3    0.492754
4    0.640777
5    0.429268
dtype: float64

In [151]:
#average rate of detecting presence of a label, by class
unique_labels_detected.sum(axis=0).divide(num_images_labeled).mean()

0.6176078294692929

In [75]:
num_images_detected = pd.DataFrame(unique_preds_by_image).sum(axis=0)
num_images_detected

0    11
1     3
2     2
3     1
4     3
5     4
dtype: int64

In [62]:
np.divide(num_images_detected, num_images_labeled)

0    0.647059
1    1.500000
2    1.000000
3    1.000000
4    1.000000
5    0.666667
dtype: float64

In [63]:
num_images_labeled.sum()

31

In [31]:
%%capture

results_folder = '/mnt/muthderd/MIDS/W210/data/val_results_3/'
data_folder = '/mnt/muthderd/MIDS/W210/data/val2017/'

#test_filenames = os.listdir(data_folder)
test_filenames = files_to_open#[:25]

all_test_annotations = pd.DataFrame(columns=['image_id', 'class_id', 'scores', 1, 2, 3, 4])

baseline_model = baseline.BaselineClassifier(model='ssd_512_resnet50_v1_coco')

pred_start = time.time()

for image in test_filenames:
    class_ids, scores, bounding_boxes = baseline_model.classify_objects(image, data_folder, results_folder)
    
    class_id_series = pd.Series(class_ids[0].asnumpy().reshape(len(class_ids[0]),))
    score_series = pd.Series(scores[0].asnumpy().reshape(len(scores[0]),))
    b_box_DF = pd.DataFrame(bounding_boxes[0].asnumpy().reshape(len(bounding_boxes[0]),4), columns=[1,2,3,4])
    
    class_id_series[class_id_series>-1]
    score_series[class_id_series>-1]
    b_box_DF[class_id_series>-1]
    
    test_annotations = b_box_DF[class_id_series>-1]
    test_annotations['class_id'] = class_id_series[class_id_series>-1]
    test_annotations['scores'] = score_series[class_id_series>-1]
    
    test_annotations['image_id'] = [image[:-4]]*len(test_annotations)
    test_annotations = test_annotations[['image_id', 'class_id', 'scores', 1, 2, 3, 4]]
    
    all_test_annotations = pd.concat([all_test_annotations, test_annotations], ignore_index=True)
    
print("Total Annotations: ", str(len(all_test_annotations)))

pred_end = time.time()
pred_duration = pred_end - pred_start

In [32]:
pred_duration

564.4249608516693

In [33]:
all_test_annotations[:10]

,image_id,class_id,scores,1,2,3,4
0,000000464476,62.0,0.994084,377.621948,288.606720,652.204346,508.373383
1,000000464476,0.0,0.991716,69.142776,77.343102,159.968185,331.558929
2,000000464476,62.0,0.315119,367.770813,51.634697,624.718262,280.566498
3,000000464476,6.0,0.197120,195.155273,32.164742,373.132996,200.863678
4,000000464476,0.0,0.105416,527.026184,351.558197,537.178528,362.751556
5,000000464476,0.0,0.098738,520.610962,352.366943,528.946655,362.981628
6,000000464476,0.0,0.096872,530.714355,351.537720,541.468994,362.626953
7,000000464476,0.0,0.090418,548.727539,353.481445,557.438843,363.910278
8,000000464476,62.0,0.084859,195.168060,31.367561,373.370697,204.342865
9,000000464476,62.0,0.081976,363.247223,109.605972,589.127808,501.856873


In [34]:
all_test_annotations.to_csv('val_result_annotations_3.csv')

In [35]:
pred_prob_threshold = 0.5
result_annotations = all_test_annotations[all_test_annotations['scores']>pred_prob_threshold]
eIDs_in_results = result_annotations.groupby('image_id')['class_id'].apply(list).to_dict()
#eIDs_in_results

In [36]:
#quick hack to deal with the fact that the class IDs predicted by the baseline model are 10 lower than those in the annotation file
#and to throw out non electronics identified
predicted_eIDs = {}
for image_id in eIDs_in_results.keys():
    corrected_IDs = np.add(eIDs_in_results[image_id],10).astype(int)
    llimit = corrected_IDs >= 72
    ulimit = corrected_IDs <=77
    #print(llimit & ulimit)
    corrected_IDs = corrected_IDs[llimit & ulimit]
    #eIDs_in_results[image_id] = list(corrected_IDs)
    predicted_eIDs[int(image_id)] = list(corrected_IDs)
#predicted_eIDs

In [37]:
total_pred_counts = [0, 0, 0, 0, 0, 0]#can use a smarter structure here
total_label_counts = [0, 0, 0, 0, 0, 0]
pred_counts = [0, 0, 0, 0, 0, 0]#can use a smarter structure here
label_counts = [0, 0, 0, 0, 0, 0]
#current counting method will not take into account false positives
for image_id in predicted_eIDs.keys():
    labels = eIDs_in_images[image_id]
    for label in labels:
        total_label_counts[label-72] += 1
    preds = predicted_eIDs[image_id]
    for pred in preds:
        total_pred_counts[pred-72] += 1
    
    unique_labels = list(set(labels))
    unique_preds = list(set(preds))
    for label in unique_labels:
        label_counts[label-72] += 1
    for pred in unique_preds:
        pred_counts[pred-72] += 1

In [38]:
total_pred_counts

[212, 190, 76, 90, 98, 113]

In [39]:
total_label_counts

[288, 230, 106, 282, 153, 259]

In [40]:
np.divide(total_pred_counts, total_label_counts)

array([0.73611111, 0.82608696, 0.71698113, 0.31914894, 0.64052288,
       0.43629344])

In [41]:
pred_counts

[172, 156, 62, 62, 78, 96]

In [42]:
label_counts

[207, 182, 88, 144, 106, 211]

In [43]:
np.divide(pred_counts, label_counts)

array([0.83091787, 0.85714286, 0.70454545, 0.43055556, 0.73584906,
       0.4549763 ])

In [44]:
sum(pred_counts)/sum(label_counts)

0.6673773987206824